# SEGUNDA VERSÃO DO MOTOR PARA GEODIFICAÇÃO
1 - Importação e aplicação das bibliotecas necessárias.

In [ ]:
import pandas as pd
import folium
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

2 - Puxando o data frame a partir de dados no formato CSV

In [ ]:
df = pd.read_csv('amostragem.csv')

3 - Criando um novo data frame para receber meus dados sem modificar os dados originais

In [ ]:
dfNew = pd.DataFrame()
colunas = ['Logradouro', 'Latitude', 'Longitude']
dfNew = pd.DataFrame(columns=colunas)

In [ ]:
dfNew['Logradouro'] = df['Logradouro']+', '+df['Municipio']+', '+df['UF']

4 - Criando as variaveis que irão pesquisar a geolocalização a partir de um endereço com um tempo limite de requisição

In [ ]:
locator = Nominatim(user_agent = 'geocode_zeta')

In [ ]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.1)

5 - Realizando a pesquisa e puxando dados geograficos de longitude e latitude a partir da pesquisa do endereço

In [ ]:
try:
    dfNew['Coordenadas'] = dfNew['Logradouro'].apply(locator.geocode)
except:
    print('Algo deu errado!')

In [ ]:
dfNew['Longitude'] = dfNew['Coordenadas'].apply(lambda x: x.longitude if x != None else None)

In [ ]:
dfNew['Latitude'] = dfNew['Coordenadas'].apply(lambda x: x.latitude if x != None else None)

In [ ]:
dfNew.isnull()

In [ ]:
dfNew.dropna(inplace=True)

In [ ]:
dfNew.to_csv('out_amostragem.csv', encoding='utf-8', index=False)

6 - Plotagem das coordenadas

In [ ]:
mapa = folium.Map(location=[lat,long],zoom_start=4)

i = 0 
for ix, ln in dfNew.iterrows():
    lat = df.Latitude[i].values
    long = df.Longitude[i].values
    try:
        folium.Marker([lat, long]).add_to(mapa)
        i += 1
    except:
        i += 1

mapa